In [ ]:
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.spatial.distance import cdist

# ChArUco 보드 설정 (전역 변수로 설정)
CHARUCO_SQUARES_X = 13
CHARUCO_SQUARES_Y = 9
CHARUCO_SQUARE_LENGTH = 0.04  # 미터 단위
CHARUCO_MARK_LENGTH = 0.02  # 미터 단위

corner_threshold = 20  # SIFT feature와 ChArUco corner 간 거리 임계값 (픽셀 단위)

def create_charuco_board():
    """ChArUco 보드 객체 생성"""
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
    board = cv2.aruco.CharucoBoard(
        (CHARUCO_SQUARES_X, CHARUCO_SQUARES_Y),
        CHARUCO_SQUARE_LENGTH,
        CHARUCO_MARK_LENGTH,
        aruco_dict
    )
    return board, aruco_dict

# 전역 보드 객체
BOARD, ARUCO_DICT = create_charuco_board()

def detect_charuco_corners_in_image(img):
    """
    이미지에서 ChArUco corner 검출 (world coordinates 매핑용)
    
    Returns:
    - corners: 검출된 corner 2D 좌표
    - ids: corner ID 
    - world_coords: 대응하는 3D world coordinates
    """
    detector = cv2.aruco.ArucoDetector(ARUCO_DICT)
    
    # ArUco 마커 검출
    marker_corners, marker_ids, _ = detector.detectMarkers(img)
    
    if len(marker_corners) == 0:
        return None, None, None
    
    # ChArUco corner 보간
    ret, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(
        marker_corners, marker_ids, img, BOARD
    )
    
    if ret == 0 or charuco_corners is None:
        return None, None, None
    
    # Corner ID로부터 world coordinates 생성
    world_coords = []
    for corner_id in charuco_ids.flatten():
        corners_per_row = CHARUCO_SQUARES_X - 1
        row = corner_id // corners_per_row
        col = corner_id % corners_per_row
        
        world_x = (col + 1) * CHARUCO_SQUARE_LENGTH
        world_y = (row + 1) * CHARUCO_SQUARE_LENGTH
        world_coords.append([world_x, world_y, 0.0])
    
    return charuco_corners.reshape(-1, 2), charuco_ids.flatten(), np.array(world_coords)

def sift_match_for_visualize(img1, img2, ratio_thresh=0.75, corner_threshold=corner_threshold):
    """
    더 관대한 버전 - corner 근처 조건을 완화
    """
    # 순수 SIFT 매칭
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    if len(kp1) == 0 or len(kp2) == 0:
        return [], None, None
    
    bf = cv2.BFMatcher()
    raw_matches = bf.knnMatch(des1, des2, k=2)
    
    sift_matches = []
    for match_pair in raw_matches:
        if len(match_pair) == 2:
            m, n = match_pair
            if m.distance < ratio_thresh * n.distance:
                pt1 = kp1[m.queryIdx].pt
                pt2 = kp2[m.trainIdx].pt
                sift_matches.append((pt1, pt2))
    
    print(f"SIFT matches (relaxed): {len(sift_matches)}")
    
    # ChArUco corner 검출
    corners1, ids1, world_coords1 = detect_charuco_corners_in_image(img1)
    corners2, ids2, world_coords2 = detect_charuco_corners_in_image(img2)
    
    if corners1 is None or corners2 is None:
        return sift_matches, None, None
    
    # 더 관대한 매칭 - corner ID 일치 조건 완화
    good_matches = []
    matched_world_coords = []
    
    for pt1, pt2 in sift_matches:
        # img1에서 가장 가까운 corner
        distances1 = np.linalg.norm(corners1 - np.array(pt1), axis=1)
        min_idx1 = np.argmin(distances1)
        
        # img2에서 가장 가까운 corner  
        distances2 = np.linalg.norm(corners2 - np.array(pt2), axis=1)
        min_idx2 = np.argmin(distances2)
        
        # 임계값 내에 있으면 매칭 (corner ID 일치 조건 제거)
        if distances1[min_idx1] < corner_threshold and distances2[min_idx2] < corner_threshold:
            good_matches.append((pt1, pt2))
            matched_world_coords.append(world_coords1[min_idx1])
    
    print(f"Relaxed matches with world coordinates: {len(good_matches)}")
    
    return good_matches, np.array(matched_world_coords) if matched_world_coords else None, None

def sift_match(img1, corner_threshold=corner_threshold):
    """
    단일 이미지에서 SIFT feature와 ChArUco corner 연결
    
    Parameters:
    - img1: 입력 이미지
    - corner_threshold: SIFT feature와 ChArUco corner 간 거리 임계값 (픽셀)
    
    Returns:
    - good_matches: [(sift_pt, charuco_corner), ...] SIFT-ChArUco 연결 쌍
    - matched_world_coords: 대응하는 3D world coordinates
    - sift_features: 모든 SIFT feature 점들 (참고용)
    """
    # SIFT feature 검출
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    
    if len(kp1) == 0:
        print("SIFT feature 검출 실패")
        return [], None, []
    
    # SIFT feature 좌표 추출
    sift_points = np.array([kp.pt for kp in kp1])
    # print(f"SIFT features detected: {len(sift_points)}")
    
    # ChArUco corner 검출
    corners1, ids1, world_coords1 = detect_charuco_corners_in_image(img1)
    
    if corners1 is None:
        print("ChArUco corner 검출 실패")
        return [], None, sift_points.tolist()
    
    # print(f"ChArUco corners detected: {len(corners1)}")
    
    # SIFT feature와 가장 가까운 ChArUco corner 연결
    good_matches = []
    matched_world_coords = []
    
    for sift_pt in sift_points:
        # 가장 가까운 ChArUco corner 찾기
        distances = np.linalg.norm(corners1 - sift_pt, axis=1)
        min_idx = np.argmin(distances)
        min_distance = distances[min_idx]
        
        # 임계값 내에 있으면 연결
        if min_distance < corner_threshold:
            charuco_corner = corners1[min_idx]
            corner_id = ids1[min_idx]
            world_coord = world_coords1[min_idx]
            
            good_matches.append((tuple(sift_pt), tuple(charuco_corner)))
            matched_world_coords.append(world_coord)
            
            # print(f"SIFT-ChArUco match: Corner ID {corner_id}, distance {min_distance:.1f}px")
    
    # print(f"Total SIFT-ChArUco matches: {len(good_matches)}")
    
    return (good_matches, 
            np.array(matched_world_coords) if matched_world_coords else None, 
            sift_points.tolist())

def draw_sift_matches(img1, img2, matches):
    """
    매칭 결과 시각화 (기존 함수 유지)
    """
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    canvas = np.zeros((max(h1, h2), w1 + w2), dtype=np.uint8)
    canvas[:h1, :w1] = img1
    canvas[:h2, w1:] = img2

    plt.figure(figsize=(15, 10))
    plt.imshow(canvas, cmap='gray')

    if len(matches) > 0:
        cmap = plt.cm.get_cmap('hsv', len(matches))
        for i, ((x1, y1), (x2, y2)) in enumerate(matches):
            color = cmap(i)
            plt.plot([x1, x2 + w1], [y1, y2], color=color, linewidth=2, marker='o', markersize=6)
    
    plt.title(f"SIFT Matches with ChArUco World Coords: {len(matches)} matches")
    plt.axis('off')
    plt.show()

IMAGE_DIR = Path('camera_images3')

img1 = cv2.imread(str(IMAGE_DIR / '0.jpeg'), cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(str(IMAGE_DIR / '1.jpeg'), cv2.IMREAD_GRAYSCALE)

matches, kp1, kp2 = sift_match_for_visualize(img1, img2)
# print(f"# of good matches: {len(matches)}")

draw_sift_matches(img1, img2, matches)

In [ ]:
def compute_homography(matches):
    A = []
    for (x, y), (xp, yp) in matches:
        A.append([-x, -y, -1, 0, 0, 0, x*xp, y*xp, xp])
        A.append([0, 0, 0, -x, -y, -1, x*yp, y*yp, yp])
    A = np.array(A)

    # SVD
    U, S, Vt = np.linalg.svd(A)
    h = Vt[-1, :]
    H = h.reshape(3, 3)
    return H / H[2, 2]

def average_reprojection_error(matches, H):
    total_error = 0
    for (x, y), (xp, yp) in matches:
        p = np.array([x, y, 1.0])
        projected = H @ p
        projected /= projected[2]
        error = np.linalg.norm(projected[:2] - np.array([xp, yp]))
        total_error += error
    return total_error / len(matches)

H = compute_homography(matches)
error = average_reprojection_error(matches, H)
print(f"Average Reprojection Error: {error:.4f} pixels")

In [ ]:
import random

def ransac_homography(matches, threshold=3.0, iterations=1000):
    best_H = None
    max_inliers = 0
    best_inliers = []

    for _ in range(iterations):
        sample = random.sample(matches, 4)
        H_candidate = compute_homography(sample)
        inliers = []
        if matches is None or len(matches) < 4:
            print("Not enough matches for RANSAC")
            continue
        else:
            for (x, y), (xp, yp) in matches:
                p = np.array([x, y, 1.0])
                projected = H_candidate @ p
                projected /= projected[2]
                error = np.linalg.norm(projected[:2] - np.array([xp, yp]))
                if error < threshold:
                    inliers.append(((x, y), (xp, yp)))

            if len(inliers) > max_inliers:
                max_inliers = len(inliers)
                best_H = H_candidate
                best_inliers = inliers

    return best_H, best_inliers

H_ransac, inlier_matches = ransac_homography(matches, threshold=5.0, iterations=1000)
error = average_reprojection_error(inlier_matches, H_ransac)

print(f"RANSAC Homography Inliers: {len(inlier_matches)} / {len(matches)}")
print(f"Average Reprojection Error after RANSAC: {error:.4f} pixels")

draw_sift_matches(img1, img2, inlier_matches)

In [ ]:
# 여러 이미지에 대해 Homography 계산
import os

filenames = sorted(os.listdir(IMAGE_DIR), key=lambda x: int(x.split('.')[0]))
H_list = []

# 기준 카메라: 0th image
img0_path = str(IMAGE_DIR / filenames[0])
img0 = cv2.imread(img0_path, cv2.IMREAD_GRAYSCALE)

for i in range(0, len(filenames)):
    img_path = str(IMAGE_DIR / filenames[i])
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    matches, kp1, kp2 = sift_match(img)
    print(f"Matching image{i}: {len(matches)} good matches")
    if len(matches) < 4:
        print(f"Not enough matches for image0 and image{i}, skipping...")
        continue
    H_ransac, inlier_matches = ransac_homography(matches, threshold=5.0, iterations=1000)
    error = average_reprojection_error(inlier_matches, H_ransac)

    H_list.append(H_ransac)

    print(f"RANSAC Homography for image{i}:")
    print(f"Inliers: {len(inlier_matches)} / {len(matches)}")
    # print(f"Homography Matrix:\n{H_ransac}\n")
    # print(f"Homography determinant: {np.linalg.det(H_ransac):.4f}")
    print(f"Average Reprojection Error: {error:.4f} pixels\n")

In [ ]:
def compute_v_ij(H, i, j):
    return np.array([
        H[0, i]*H[0, j],
        H[0, i]*H[1, j] + H[1, i]*H[0, j],
        H[1, i]*H[1, j],
        H[2, i]*H[0, j] + H[0, i]*H[2, j],
        H[2, i]*H[1, j] + H[1, i]*H[2, j],
        H[2, i]*H[2, j]
    ])

def estimate_intrinsics(H_list):
    V = []
    for H in H_list:
        v12 = compute_v_ij(H, 0, 1)
        v11 = compute_v_ij(H, 0, 0)
        v22 = compute_v_ij(H, 1, 1)
        V.append(v12)
        V.append(v11 - v22)
    V = np.array(V)

    # Solve Vb = 0 using SVD
    _, _, Vt = np.linalg.svd(V)
    b = Vt[-1, :]

    # Extract parameters from b
    B11, B12, B22, B13, B23, B33 = b
    v0 = (B12*B13 - B11*B23) / (B11*B22 - B12**2)
    lam = B33 - (B13**2 + v0*(B12*B13 - B11*B23)) / B11
    # print(f'lam: {lam}, B11: {B11}, (B11*B22 - B12**2): {B11*B22 - B12**2}, lab* B11: {lam / B11}')
    alpha = np.sqrt(lam / B11)
    beta = np.sqrt(lam * B11 / (B11*B22 - B12**2))
    gamma = -B12 * alpha**2 * beta / lam
    u0 = gamma * v0 / beta - B13 * alpha**2 / lam

    # Construct K matrix
    K = np.array([
        [alpha, gamma, u0],
        [0,     beta,  v0],
        [0,     0,     1]
    ])
    return K

def estimate_extrinsics(H, K):
    K_inv = np.linalg.inv(K)
    h1 = H[:, 0]
    h2 = H[:, 1]
    h3 = H[:, 2]

    lambda_ = 1.0 / np.linalg.norm(K_inv @ h1)
    r1 = lambda_ * (K_inv @ h1)
    r2 = lambda_ * (K_inv @ h2)
    r3 = np.cross(r1, r2)
    t = lambda_ * (K_inv @ h3)

    # Ensure R is a valid rotation matrix using orthonormalization (SVD)
    R = np.stack([r1, r2, r3], axis=1)
    U, _, Vt = np.linalg.svd(R)
    R_orthonormal = U @ Vt

    return R_orthonormal, t

def is_valid_homography(H, cond_thresh=1e6):
    return (
        np.isfinite(H).all() and 
        np.abs(H[2, 2] - 1.0) < 1e-3 and 
        np.linalg.cond(H) < cond_thresh
    )

def estimate_extrinsics_safe(H_list, K, cond_thresh=1e8):
    extrinsics = []
    for i, H in enumerate(H_list):
        if not is_valid_homography(H, cond_thresh):
            print(f"⚠️ Skipping H[{i}] due to high condition number: {np.linalg.cond(H):.2e}")
            continue
        try:
            R, t = estimate_extrinsics(H, K)
            if R is not None:
                extrinsics.append((R, t))
        except np.linalg.LinAlgError:
            print(f"⚠️ SVD failed for H[{i}]")
    return extrinsics

K = estimate_intrinsics(H_list)
print("Estimated Intrinsic Matrix K:\n", K)

# First camera's extrinsics (identity for the first camera)
extrinsics = [(np.eye(3), np.zeros(3))]

# Append the extrinsics for the other cameras
for idx, H in enumerate(H_list):
    # if idx == 0:
    #     continue  # Skip the first camera
    R, t = estimate_extrinsics(H, K)
    print(f"Image {idx+1}:")
    print("Rotation Matrix R:\n", R)
    print("Translation Vector t:\n", t)
    print()
    extrinsics.append((R, t))


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_camera_frustum(ax, R, camera_center, scale):
    """
    카메라 프러스텀(시야각)을 시각화하는 함수
    """
    # 프러스텀 크기
    frustum_length = scale * 1.5
    frustum_width = scale * 0.8
    frustum_height = scale * 0.6
    
    # 프러스텀 모서리 점들 (카메라 좌표계에서)
    frustum_points_local = np.array([
        [0, 0, 0],  # 카메라 중심
        [-frustum_width, -frustum_height, frustum_length],  # 좌하
        [frustum_width, -frustum_height, frustum_length],   # 우하
        [frustum_width, frustum_height, frustum_length],    # 우상
        [-frustum_width, frustum_height, frustum_length]    # 좌상
    ])
    
    # 월드 좌표계로 변환
    frustum_points_world = []
    for point in frustum_points_local:
        world_point = R @ point + camera_center
        frustum_points_world.append(world_point)
    
    frustum_points_world = np.array(frustum_points_world)
    
    # 프러스텀 선 그리기
    # 카메라 중심에서 각 모서리로
    for i in range(1, 5):
        ax.plot([frustum_points_world[0, 0], frustum_points_world[i, 0]],
                [frustum_points_world[0, 1], frustum_points_world[i, 1]],
                [frustum_points_world[0, 2], frustum_points_world[i, 2]], 
                'gray', alpha=0.4, linewidth=1)
    
    # 프러스텀 사각형 그리기
    for i in range(1, 5):
        next_i = i + 1 if i < 4 else 1
        ax.plot([frustum_points_world[i, 0], frustum_points_world[next_i, 0]],
                [frustum_points_world[i, 1], frustum_points_world[next_i, 1]],
                [frustum_points_world[i, 2], frustum_points_world[next_i, 2]], 
                'gray', alpha=0.4, linewidth=1)

# Visualize camera extrinsics in 3D space
def plot_camera_poses(K, extrinsics):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # World coordinate axes
    ax.quiver(0, 0, 0, 1, 0, 0, color='r', length=0.5)
    ax.quiver(0, 0, 0, 0, 1, 0, color='g', length=0.5)
    ax.quiver(0, 0, 0, 0, 0, 1, color='b', length=0.5)

    for idx, (R, t) in enumerate(extrinsics):
        # Camera center in world coordinates
        C = -R.T @ t

        # Draw camera axis
        cam_axis = R.T
        ax.quiver(C[0], C[1], C[2], cam_axis[0, 0], cam_axis[1, 0], cam_axis[2, 0], color='r', length=0.3)
        ax.quiver(C[0], C[1], C[2], cam_axis[0, 1], cam_axis[1, 1], cam_axis[2, 1], color='g', length=0.3)
        ax.quiver(C[0], C[1], C[2], cam_axis[0, 2], cam_axis[1, 2], cam_axis[2, 2], color='b', length=0.3)

        ax.text(C[0], C[1], C[2], f"Cam{idx+1}", color='black')

        plot_camera_frustum(ax, R, C, scale=500)

    ax.set_title("Estimated Camera Poses")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.view_init(elev=30, azim=-60)
    plt.tight_layout()
    plt.show()

plot_camera_poses(K, extrinsics)